In [1]:

from sandbox.specs import (
    MAP_REPO_TO_REQS_PATHS,
    MAP_REPO_VERSION_TO_SPECS_PY,
)
from sandbox import bash_session, docker_build

In [2]:
from datasets import load_dataset

dataset = load_dataset("lahirum/SWE_Experimental", split="train")

/home/lahiru-menik/miniconda3/envs/agentless/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MAP_REPO_VERSION_TO_SPECS_PY[dataset[0]['repo']][dataset[0]['version']]

{'python': '3.9',
 'packages': 'requirements.txt',
 'install': 'python -m pip install -e .',
 'test_cmd': './tests/runtests.py --verbosity 2'}

In [1]:
import docker

def image_exists(name):
    # client = docker.from_env()
    client = docker.DockerClient(base_url='unix:///home/lahiru-menik/.docker/desktop/docker.sock')
    # print(client.images.list(), "lll")
    try:
        client.images.get(name)
        return True
    except docker.errors.ImageNotFound:
        return False
    except docker.errors.APIError as e:
        print(f"Docker API error: {e}")
        return False

In [2]:
print(image_exists("django_4.1"))

True


In [6]:

client = docker.DockerClient(base_url='unix:///home/lahiru-menik/.docker/desktop/docker.sock')
images = client.images.list()
print(images)

[<Image: 'django_4.1:latest'>, <Image: 'django-env:4.2'>, <Image: 'swe/310:latest'>, <Image: 'httpd:2.4'>, <Image: 'hello-world:latest'>]


In [7]:
import docker

client = docker.from_env()
images = client.images.list()

print("=== Docker SDK sees the following images ===")
for image in images:
    print(image.id)
    print("Tags:", image.tags)
    print("-----")

=== Docker SDK sees the following images ===
sha256:74cc54e27dc41bb10dc4b2226072d469509f2f22f1a3ce74f4a59661a1d44602
Tags: ['hello-world:latest']
-----


In [8]:
import docker
client = docker.from_env()
print("Docker SDK connected to:", client.api.base_url)


Docker SDK connected to: http+docker://localhost


In [9]:
import ast

# Suppose this is your string (from dataset[0]['FAIL_TO_PASS'])
fail_to_passs = '["test_urlfield_clean_invalid (forms_tests.field_tests.test_urlfield.URLFieldTest)", "test_urlfield_clean_not_required (forms_tests.field_tests.test_urlfield.URLFieldTest)"]'

# Convert the string to a real list
fail_list = ast.literal_eval(fail_to_passs)

print(fail_list)
print(type(fail_list))  # should show <class 'list'>


['test_urlfield_clean_invalid (forms_tests.field_tests.test_urlfield.URLFieldTest)', 'test_urlfield_clean_not_required (forms_tests.field_tests.test_urlfield.URLFieldTest)']
<class 'list'>


In [10]:
import re
import ast
fail_to_passs = dataset[0]['FAIL_TO_PASS']
fail_to_passs = ast.literal_eval(fail_to_passs)


fail_to_passs= {
    re.sub(r'\s*\(.*\)', '', item): re.search(r'\((.*?)\)', item).group(1)
    for item in fail_to_passs
}

print(fail_to_passs)


{'test_urlfield_clean_invalid': 'forms_tests.field_tests.test_urlfield.URLFieldTest', 'test_urlfield_clean_not_required': 'forms_tests.field_tests.test_urlfield.URLFieldTest'}


In [13]:
from commit import update
import os
import tempfile
from pathlib import Path
import pexpect

for i in range(1):
    commit_id = dataset[i]['base_commit']
    fail_to_passs = dataset[i]['FAIL_TO_PASS']
    fail_to_passs = ast.literal_eval(fail_to_passs)


    fail_to_passs= {
        re.sub(r'\s*\(.*\)', '', item): re.search(r'\((.*?)\)', item).group(1)
        for item in fail_to_passs
    }

    version = dataset[i]['version']
    pass_to_pass = dataset[i]['PASS_TO_PASS']
    name = dataset[i]['instance_id'].split("__")[0]
    # print(name)
    update(name, commit_id)
    bash = bash_session.BashSession()
    bash.run_command(f"cp -r {name} testbed")
    host_dir = os.path.abspath(".") + "/testbed/" + name
    bash.logfile = open(f"{name}_{version}_dockerbuild.log", "w", encoding="utf-8")
    container_dir = "/app" 
    exist_check = image_exists(name + "_" + version)
    print(exist_check)
    specs = MAP_REPO_VERSION_TO_SPECS_PY[dataset[i]['repo']][dataset[i]['version']]
    # exist_check = bash.run_command(f"docker images | grep -q '^{name+'_'+version}' && echo yes || echo no")
    if not exist_check:
        print(f"Building image {name}_{version}...")
        if "pip_packages" in specs:
            pip_packages = " ".join(specs["pip_packages"])
        else:
            pip_packages = ""
        if "pre_install" in specs:
            pre_install_commands = "\n".join(specs["pre_install"])
        else:
            pre_install_commands = ""
        dockerfile_str = docker_build.generate_dockerfile(
            python_version= specs["python"],
            pre_install= pre_install_commands,
            pip_packages=pip_packages,
            
        )
        
        tmpdir = tempfile.mkdtemp()
        Path(f"{tmpdir}/Dockerfile").write_text(dockerfile_str)
        print(f"Generated Dockerfile for {name}_{version}")
        bash.run_command(f"docker build -t {name}_{version} {tmpdir}")

      
    bash.run_command(f"docker run --mount type=bind,src={host_dir},dst={container_dir} -it {name}_{version} bash")
    if "install" in specs:
        bash.run_command(specs["install"])
    if "packages" in specs:
        if not specs["packages"].startswith("requiremen"):
            bash.run_command(f"pip install {specs['packages']}")
        else:
            if dataset[i]['repo'] in MAP_REPO_TO_REQS_PATHS:
                requirements_path = MAP_REPO_TO_REQS_PATHS[dataset[i]['repo']]
                requirements_path = " ".join(requirements_path)
            else:
                requirements_path = specs['packages']
            bash.run_command(f"pip install -r {requirements_path}")

    # for test_cm in fail_to_passs.values():
    #     print(f"Running test: {test_cm}")
    #     print(bash.run_command(f"{specs['test_cmd']} {test_cm}"))
    print(bash.before)
    bash.close()
        
        
    
    
    

django
DiGraph with 27538 nodes and 40138 edges
Checked out to 4fd3044ca0135da903a70dfb66992293f529ecf1
DiGraph with 27538 nodes and 40138 edges
False
Building image django_4.1...
Generated Dockerfile for django_4.1


AttributeError: 'BashSession' object has no attribute 'before'